Python notebook developed by:
Rodrigo Cedeno & Emanuele Capizzi
For project ...

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
import matplotlib.figure
import fiona.crs
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
import xarray as xr
import time
import yaml
import uuid
import os
from glob import iglob # data access in file manager
import rasterio
from os.path import join # same
import plotly.graph_objects as go
import rioxarray as rio

#Hide Warnings
import warnings
warnings.filterwarnings('ignore')

#Progress bar
import ipywidgets as widgets
from termcolor import colored # Print colored text

In [2]:
file_basepath = '/home/asi/data/classified_images'
#search inside sub folders
list_files = sorted(list(iglob(join(file_basepath, '*.tif*'), recursive=True)))
#without searching inside sub folders
#list_files = sorted(list(iglob(join(files_to_move, '*.tif'), recursive=True)))
list_files

['/home/asi/data/classified_images/classified_RandomForest_20230209_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230322_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230408_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230617_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230710_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230808_medianfilter_30m.tif']

In [3]:
file_names = []
for path in list_files:
    # Extract the last portion after the last slash (/)
    file_name = path.split('/')[-1]
    file_name = file_name.replace('_RandomForest', '')
    file_name = file_name.replace('_medianfilter_30m.tif', '')
    file_names.append(file_name)
print(file_names)

['classified_20230209', 'classified_20230322', 'classified_20230408', 'classified_20230617', 'classified_20230710', 'classified_20230808']


In [4]:
file_dict = {}
for key in file_names:
    for value in list_files:
        file_dict[key] = value
        list_files.remove(value)
        break
file_dict

{'classified_20230209': '/home/asi/data/classified_images/classified_RandomForest_20230209_medianfilter_30m.tif',
 'classified_20230322': '/home/asi/data/classified_images/classified_RandomForest_20230322_medianfilter_30m.tif',
 'classified_20230408': '/home/asi/data/classified_images/classified_RandomForest_20230408_medianfilter_30m.tif',
 'classified_20230617': '/home/asi/data/classified_images/classified_RandomForest_20230617_medianfilter_30m.tif',
 'classified_20230710': '/home/asi/data/classified_images/classified_RandomForest_20230710_medianfilter_30m.tif',
 'classified_20230808': '/home/asi/data/classified_images/classified_RandomForest_20230808_medianfilter_30m.tif'}

In [5]:
def get_key_from_value(dictionary, target_value):
    matching_keys = [key for key, value in dictionary.items() if value == target_value]
    return matching_keys[0] if matching_keys else None

key = get_key_from_value(file_dict, '/home/asi/data/classified_images/classified_RandomForest_20230209_medianfilter_30m.tif')
key

'classified_20230209'

In [6]:
list_files = sorted(list(iglob(join(file_basepath, '*.tif*'), recursive=True)))
list_files

['/home/asi/data/classified_images/classified_RandomForest_20230209_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230322_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230408_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230617_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230710_medianfilter_30m.tif',
 '/home/asi/data/classified_images/classified_RandomForest_20230808_medianfilter_30m.tif']

# Generate product measurements list

In [7]:
f = widgets.IntProgress(min=0, max=1, description='Processing images', bar_style='info')
display(f)
p_name = "classified_images"
text_p0 = """name: """ + p_name + """
description: Classified images using scikit-learn Random Forest Classified
metadata_type: eo3

metadata:
    product:
        name: """ + p_name + """
        format: GeoTIFF

storage:
    crs: EPSG:32632
    dimension_order: [time, x, y]
    resolution: 
        x: 30 
        y: -30"""

file = list_files[0]
date_uuid = file[file.find('202'):file.find('202')+8]
src = rasterio.open(file) 
xmin,ymin,xmax,ymax = src.bounds
affine = src.transform
shape = src.shape
text_p2_1 = """"""
text_p2 = """"""
text_p1 = ("""

measurements:""")
""""""

text_p2_1 = """
    - name: classified_images
      dtype: float32
      nodata: NaN
      units: ''
    """
text_p2 = text_p2 + text_p2_1
text_p3 = text_p0 + text_p1 + text_p2
yaml_text = open('/home/asi/datacube_conf/'+p_name+'.yaml', "wt")
yaml_text.write(text_p3)
yaml_text.close()
os.system("""cd /home/asi/datacube_conf; 
datacube product add /home/asi/datacube_conf/"""+p_name+""".yaml""")
#os.system("""cd /home/asi/datacube_conf; 
#datacube product update --allow-unsafe /home/asi/datacube_conf/"""+p_name+""".yaml""")
f.value += 1
        
print(colored('All classified images products have been updated into the ODC','green'))

IntProgress(value=0, bar_style='info', description='Processing images', max=1)

Adding "classified_images" (this might take a while) DONE


2023-11-16 10:01:22,671 23763 datacube.index.postgres._products WARNING Product classified_images is already in the database, checking for differences


All classified images products have been updated into the ODC


## Generate and Register Dataset 

In [8]:
f = widgets.IntProgress(min=0, max=len(list_files), description='Processing images', bar_style='info')
display(f)

for file in list_files:
    date_uuid = file[file.find('202'):file.find('202')+8]
    print(date_uuid)
    src = rasterio.open(file)
    xmin,ymin,xmax,ymax = src.bounds
    affine = src.transform
    shape = src.shape
    text_p1 = ("""$schema: https://schemas.opendatacube.org/dataset

id: 60000000-0000-0000-0000-0000"""+ str(date_uuid) +"""

product:
  name: classified_images
  href: None
  format: GeoTIFF

crs: EPSG:32632

geometry:
  type: Polygon
  coordinates: [[["""+str(xmin)+""", """+str(ymin)+"""], ["""+str(xmin)+""", """+str(ymax)+"""], ["""+str(xmax)+""", """+str(ymax)+"""], ["""+str(xmax)+""", """+str(ymin)+"""], ["""+str(xmin)+""", """+str(ymin)+"""]]]

grids:
  default:
    shape: ["""+str(shape[0])+""","""+str(shape[1])+"""]
    transform: ["""+str(affine[0])+""","""+str(affine[1])+""","""+str(affine[2])+""","""+str(affine[3])+""","""+str(affine[4])+""","""+str(affine[5])+""", 0,0,1]

lineage: {}
measurements:
""")
    text_p2 = """"""

    text_p2_1 = """
  classified_images:
    path: """+str(file)+""""""
    text_p2 = text_p2 + text_p2_1
    
    text_p3 = """

properties:
  odc:file_format: GeoTIFF
  datetime: """+str(date_uuid)+"""T10:30:00Z
    """
    yaml_text = open(file.replace('.tif', '.yaml'), "wt")
    yaml_text.write(text_p1+text_p2+text_p3)
    yaml_text.close()
    os.system("""cd /home/asi/datacube_conf; 
    datacube dataset add """+file.replace('.tif', '.yaml'))
    
    f.value += 1
        
print(colored('All classified image products have been imported into the ODC','green'))

IntProgress(value=0, bar_style='info', description='Processing images', max=6)

20230209


2023-11-16 10:01:29,863 23770 datacube.index.postgres._datasets WARNING Dataset 60000000-0000-0000-0000-000020230209 is already in the database


20230322


2023-11-16 10:01:31,633 23777 datacube.index.postgres._datasets WARNING Dataset 60000000-0000-0000-0000-000020230322 is already in the database


...

20230710


2023-11-16 10:01:37,076 23798 datacube.index.postgres._datasets WARNING Dataset 60000000-0000-0000-0000-000020230710 is already in the database


20230808


2023-11-16 10:01:38,740 23805 datacube.index.postgres._datasets WARNING Dataset 60000000-0000-0000-0000-000020230808 is already in the database


All classified image products have been imported into the ODC
